In [1]:
import numpy as np
import os
import tensorflow as tf
from IPython.display import clear_output, Image, display, HTML
from tabulate import tabulate
###### Do not modify here ###### 
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = graph_def
    #strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

###### Do not modify here ###### 



Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


## No dropout

### Define Graph

In [8]:
def fully_connected_layer(x, dims, activate, name, mode=False, dropout=None):
    '''
    [Function Input]
    x        "Tensor" : The input tensor
    dims     "List"   : The dimension of Input and Output
    activate "String" : The name of activate function
    name     "String" : The name of this operation
    
    [Function Output]
    The Result of Linear Function + Activate Function 
    '''
    with tf.variable_scope(name):
#         W = tf.get_variable("Weights", dims, initializer=tf.contrib.layers.variance_scaling_initializer()) # Initialize Weight with He
#         b = tf.get_variable("Bias", [1], initializer=tf.constant_initializer(0)) # Initialize Bias with constant
#         output = tf.matmul(x, W) + b # Connect each elements for model
        
        # Define Activate Function
        if activate == 'elu':
#             output = tf.nn.elu(output, name="Elu")
            activation = tf.nn.elu
        elif activate == 'softmax':
#             output = tf.nn.softmax(output, name="Softmax")
            activation = tf.nn.softmax
        
        #Build Layer
        output = tf.layers.dense(inputs=x, 
                                 units=dims[1], 
                                 activation=activation, 
                                 kernel_initializer=tf.contrib.layers.variance_scaling_initializer(),
                                 name="Neural_Operation"
                                )  # Layer using dense
        if dropout:
            output = tf.layers.dropout(output, rate=dropout, training=mode, name="Dropout")
    return output


# Clear graph
tf.reset_default_graph()

# Define Input Entry
x = tf.placeholder(tf.float32, shape=(None, 784), name="Input_X") # Input
y = tf.placeholder(tf.int32, shape=(None,), name ="Input_Label")  # Labels
mode = tf.placeholder(tf.bool, name ="Mode")  # Train or Inference Mode

# Define Connect Layers
x1 = fully_connected_layer(x , [784, 128], "elu", "Fully_Connected_Layer_1", mode) # 1st 128 nodes Layer
x2 = fully_connected_layer(x1, [128, 128], "elu", "Fully_Connected_Layer_2", mode) # 2nd 128 nodes Layer
x3 = fully_connected_layer(x2, [128, 128], "elu", "Fully_Connected_Layer_3", mode) # 3rd 128 nodes Layer
x4 = fully_connected_layer(x3, [128, 128], "elu", "Fully_Connected_Layer_4", mode) # 4th 128 nodes Layer
x5 = fully_connected_layer(x4, [128, 128], "elu", "Fully_Connected_Layer_5", mode) # 5th 128 nodes Layer
y_ = fully_connected_layer(x5, [128, 5], "softmax", "Softmax_Layer")         # 128 to 5 class with softmax

# Define Loss Function
cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_, name="Cross_Entropy"))

# Define Training Process
train_step = tf.train.AdamOptimizer(0.0017).minimize(cross_entropy)

# Define Accuracy
predicted_class = tf.argmax(y_,1, output_type=tf.int32)
correct_predict = tf.equal(y, predicted_class) # [True, False ..., True]
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float32)) # [True, False ..., True] --> [1,0,...,1]

# Define Precision
precision = tf.metrics.precision(y, predicted_class)
"""
GET TP + FP
1. get the indices of predicted class that is specific label(ex:0)

GET TP/(TP + FP)
2. get the correspondind ground true label from indices 
3. check if they are also match the label
4. take average
"""
prec_zer = tf.reduce_mean(tf.cast(tf.equal(0, tf.gather(y , tf.where(tf.equal(0, predicted_class)))), tf.float32))
prec_one = tf.reduce_mean(tf.cast(tf.equal(1, tf.gather(y , tf.where(tf.equal(1, predicted_class)))), tf.float32))
prec_two = tf.reduce_mean(tf.cast(tf.equal(2, tf.gather(y , tf.where(tf.equal(2, predicted_class)))), tf.float32))
prec_thi = tf.reduce_mean(tf.cast(tf.equal(3, tf.gather(y , tf.where(tf.equal(3, predicted_class)))), tf.float32))
prec_fou = tf.reduce_mean(tf.cast(tf.equal(4, tf.gather(y , tf.where(tf.equal(4, predicted_class)))), tf.float32))


# Define Recall
recall = tf.metrics.recall(y, predicted_class)
"""
GET TP + FN
1. get the indices of ground true label that is specific label(ex:0)

GET TP/(TP + FN)
2. get the correspondind predicted class from indices 
3. check if they are also match the label
4. take average
"""
recal_zer = tf.reduce_mean(tf.cast(tf.equal(0, tf.gather(predicted_class , tf.where(tf.equal(0, y)))), tf.float32))
recal_one = tf.reduce_mean(tf.cast(tf.equal(1, tf.gather(predicted_class , tf.where(tf.equal(1, y)))), tf.float32))
recal_two = tf.reduce_mean(tf.cast(tf.equal(2, tf.gather(predicted_class , tf.where(tf.equal(2, y)))), tf.float32))
recal_thi = tf.reduce_mean(tf.cast(tf.equal(3, tf.gather(predicted_class , tf.where(tf.equal(3, y)))), tf.float32))
recal_fou = tf.reduce_mean(tf.cast(tf.equal(4, tf.gather(predicted_class , tf.where(tf.equal(4, y)))), tf.float32))


In [9]:
with tf.Session() as sess:
    show_graph(tf.get_default_graph().as_graph_def())

In [10]:
# prepare the training batch
train_data = tf.contrib.data.Dataset.from_tensor_slices((X_train1,y_train1)).batch(2048).repeat()
train_iterator = train_data.make_one_shot_iterator() # Create an iterator to go through the training data
train_next_batch = train_iterator.get_next()

saver = tf.train.Saver() # to store the model

# Model Hyper Parameter
epochs_num = 3000
best_accuracy = 0.8
best_epochs = 0
testing_accuracy = None
continuous_fail_count = 0
table = []
test_table = []
labels = ["0","1","2","3","4"]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) # initialize the weights 
    sess.run(tf.local_variables_initializer()) # initialize the local variables hidden in the tf.metrics.recallmethod.

    for epochs in range(epochs_num): # starting the training process and set the epochs_num
        train, label = sess.run(train_next_batch) # Get the mini-batch data sample
        sess.run(train_step, feed_dict={x:train, y:label, mode:True}) # Feed the features, labe, training_mode  to network to train
        
        # Get best accuracy
        if epochs % 100 == 0: # validate for every 100 epochs
            res = sess.run([accuracy, precision, recall, prec_zer,prec_one,prec_two,prec_thi,prec_fou,recal_zer,recal_one,recal_two,recal_thi,recal_fou], feed_dict={x:X_valid1, y:y_valid1, mode:False})
            valid_accuracy = res[0]
            
            if valid_accuracy > best_accuracy: #checking current validation result better than current best one
                
                best_accuracy = valid_accuracy # Record the best accuracy
                best_precision = res[1][0] # Record the overall precision
                best_recall = res[2][0]# Record the best overall recall
                
                # organize each column of row of report table for validation set
                table = [] # reset table
                for i in range(5): # start the loop to get all the validation result
                    table_row = [labels[i]] # set the value of label column
                    table_row.append(res[i+3]) # set the value of precision column
                    table_row.append(res[i*2+3]) # set the value of recall column
                    table.append(table_row) # append to report table
                
                # organize each column of row of report table for testing set
                res = sess.run([accuracy, precision, recall, prec_zer,prec_one,prec_two,prec_thi,prec_fou,recal_zer,recal_one,recal_two,recal_thi,recal_fou], feed_dict={x:X_test1, y:y_test1, mode:False})
                testing_accuracy = res[0]
                # organize each column of row of report table
                test_table = [] # reset table
                for i in range(5): # start the loop to get all the validation result
                    table_row = [labels[i]] # set the value of label column
                    table_row.append(res[i+3]) # set the value of precision column
                    table_row.append(res[i*2+3]) # set the value of recall column
                    test_table.append(table_row) # append to report table
                
                print("Best epochs:{} \n\tTrain Accuracy:{}\n\tValid Accuracy:{}\n\tTest  Accuracy:{}\n".format(
                        epochs+1,
                        sess.run(accuracy, feed_dict={x:train, y:label, mode:False}),
                        valid_accuracy,
                        testing_accuracy)
                    ) # print the validation result, which contain current epochs, training set accuracy, validation set accuracy, testing set accuracy
                best_epochs = epochs+1 # Record the best epochs
                save_path = saver.save(sess, "./Models/Team26_HW2.ckpt") #keep the best model
                continuous_fail_count = 0 # reset the fail count
                
            else: # every time the current validation result is not better than the current best valid accuracy
                continuous_fail_count += 1 # we increment the fail count by 1
                if continuous_fail_count > 5: # if total count larger then 5, we consider the model wont impove anymore
                    print("Early Stop! Best batches:{}, Total batches:{}".format(best_epochs,epochs+1)) # print the alert message
                    break  # stop the training step
    
    # Print the finall model performance
    print("Final Report\n")
    print("Validation Result")
    print(tabulate(table, headers=["Label", "Precision", "Recall"]))
    print("Average Accuracy:{}\n".format(best_accuracy))
    
    # render testing performance
    print("Testing Result\n")
    print(tabulate(test_table, headers=["Label", "Precision", "Recall"]))
    print("Average Accuracy:{}\n".format(testing_accuracy))

Best epochs:101 
	Train Accuracy:0.98583984375
	Valid Accuracy:0.9796715974807739
	Test  Accuracy:0.9824868440628052

Best epochs:201 
	Train Accuracy:0.9892578125
	Valid Accuracy:0.9859265089035034
	Test  Accuracy:0.9857949018478394

Best epochs:301 
	Train Accuracy:0.99072265625
	Valid Accuracy:0.9874902367591858
	Test  Accuracy:0.9889083504676819

Best epochs:501 
	Train Accuracy:0.998046875
	Valid Accuracy:0.989835798740387
	Test  Accuracy:0.9904650449752808

Best epochs:601 
	Train Accuracy:0.99951171875
	Valid Accuracy:0.9917904734611511
	Test  Accuracy:0.9922164082527161

Best epochs:701 
	Train Accuracy:0.9990234375
	Valid Accuracy:0.9921814203262329
	Test  Accuracy:0.9935784935951233

Early Stop! Model seems not improve any more..
Final Report

Validation Result
  Label    Precision    Recall
-------  -----------  --------
      0     0.995833  0.995833
      1     0.987654  0.991736
      2     0.991736  0.992551
      3     0.993878  0.994671
      4     0.992551  0.98783
Av

## Dropout

### Define graph

In [49]:
dropout = 0.1

In [50]:
# Clear graph
tf.reset_default_graph()

# Define Input Entry
x = tf.placeholder(tf.float32, shape=(None, 784), name="Input_X") # Input
y = tf.placeholder(tf.int32, shape=(None,), name ="Input_Label")  # Labels
mode = tf.placeholder(tf.bool, name ="Mode")  # Train or Inference Mode

# Define Connect Layers
x1 = fully_connected_layer(x , [784, 128], "elu", "Fully_Connected_Layer_1", mode, dropout) # 1st 128 nodes Layer
x2 = fully_connected_layer(x1, [128, 128], "elu", "Fully_Connected_Layer_2", mode, dropout) # 2nd 128 nodes Layer
x3 = fully_connected_layer(x2, [128, 128], "elu", "Fully_Connected_Layer_3", mode, dropout) # 3rd 128 nodes Layer
x4 = fully_connected_layer(x3, [128, 128], "elu", "Fully_Connected_Layer_4", mode, dropout) # 4th 128 nodes Layer
x5 = fully_connected_layer(x4, [128, 128], "elu", "Fully_Connected_Layer_5", mode, dropout) # 5th 128 nodes Layer
y_ = fully_connected_layer(x5, [128, 5], "softmax", "Softmax_Layer")         # 128 to 5 class with softmax

# Define Loss Function
cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_, name="Cross_Entropy"))

# Define Training Process
train_step = tf.train.AdamOptimizer(0.0017).minimize(cross_entropy)

# Define Accuracy
predicted_class = tf.argmax(y_,1, output_type=tf.int32)
correct_predict = tf.equal(y, predicted_class) # [True, False ..., True]
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float32)) # [True, False ..., True] --> [1,0,...,1]

# Define Precision
precision = tf.metrics.precision(y, predicted_class)
"""
GET TP + FP
1. get the indices of predicted class that is specific label(ex:0)

GET TP/(TP + FP)
2. get the correspondind ground true label from indices 
3. check if they are also match the label
4. take average
"""
prec_zer = tf.reduce_mean(tf.cast(tf.equal(0, tf.gather(y , tf.where(tf.equal(0, predicted_class)))), tf.float32))
prec_one = tf.reduce_mean(tf.cast(tf.equal(1, tf.gather(y , tf.where(tf.equal(1, predicted_class)))), tf.float32))
prec_two = tf.reduce_mean(tf.cast(tf.equal(2, tf.gather(y , tf.where(tf.equal(2, predicted_class)))), tf.float32))
prec_thi = tf.reduce_mean(tf.cast(tf.equal(3, tf.gather(y , tf.where(tf.equal(3, predicted_class)))), tf.float32))
prec_fou = tf.reduce_mean(tf.cast(tf.equal(4, tf.gather(y , tf.where(tf.equal(4, predicted_class)))), tf.float32))


# Define Recall
recall = tf.metrics.recall(y, predicted_class)
"""
GET TP + FN
1. get the indices of ground true label that is specific label(ex:0)

GET TP/(TP + FN)
2. get the correspondind predicted class from indices 
3. check if they are also match the label
4. take average
"""
recal_zer = tf.reduce_mean(tf.cast(tf.equal(0, tf.gather(predicted_class , tf.where(tf.equal(0, y)))), tf.float32))
recal_one = tf.reduce_mean(tf.cast(tf.equal(1, tf.gather(predicted_class , tf.where(tf.equal(1, y)))), tf.float32))
recal_two = tf.reduce_mean(tf.cast(tf.equal(2, tf.gather(predicted_class , tf.where(tf.equal(2, y)))), tf.float32))
recal_thi = tf.reduce_mean(tf.cast(tf.equal(3, tf.gather(predicted_class , tf.where(tf.equal(3, y)))), tf.float32))
recal_fou = tf.reduce_mean(tf.cast(tf.equal(4, tf.gather(predicted_class , tf.where(tf.equal(4, y)))), tf.float32))

In [31]:
# prepare the training batch
train_data = tf.contrib.data.Dataset.from_tensor_slices((X_train1,y_train1)).batch(2048).repeat()
train_iterator = train_data.make_one_shot_iterator() # Create an iterator to go through the training data
train_next_batch = train_iterator.get_next()

saver = tf.train.Saver() # to store the model

# Model Hyper Parameter
epochs_num = 3000
best_accuracy = 0.8
best_epochs = 0
testing_accuracy = None
continuous_fail_count = 0
table = []
test_table = []
labels = ["0","1","2","3","4"]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) # initialize the weights 
    sess.run(tf.local_variables_initializer()) # initialize the local variables hidden in the tf.metrics.recallmethod.

    for epochs in range(epochs_num): # starting the training process and set the epochs_num
        train, label = sess.run(train_next_batch) # Get the mini-batch data sample
        sess.run(train_step, feed_dict={x:train, y:label, mode:True}) # Feed the features, labe, training_mode  to network to train
        
        # Get best accuracy
        if epochs % 100 == 0: # validate for every 100 epochs
            res = sess.run([accuracy, precision, recall, prec_zer,prec_one,prec_two,prec_thi,prec_fou,recal_zer,recal_one,recal_two,recal_thi,recal_fou], feed_dict={x:X_valid1, y:y_valid1, mode:False})
            valid_accuracy = res[0]
            
            if valid_accuracy > best_accuracy: #checking current validation result better than current best one
                
                best_accuracy = valid_accuracy # Record the best accuracy
                best_precision = res[1][0] # Record the overall precision
                best_recall = res[2][0]# Record the best overall recall
                
                # organize each column of row of report table for validation set
                table = [] # reset table
                for i in range(5): # start the loop to get all the validation result
                    table_row = [labels[i]] # set the value of label column
                    table_row.append(res[i+3]) # set the value of precision column
                    table_row.append(res[i*2+3]) # set the value of recall column
                    table.append(table_row) # append to report table
                
                # organize each column of row of report table for testing set
                res = sess.run([accuracy, precision, recall, prec_zer,prec_one,prec_two,prec_thi,prec_fou,recal_zer,recal_one,recal_two,recal_thi,recal_fou], feed_dict={x:X_test1, y:y_test1, mode:False})
                testing_accuracy = res[0]
                # organize each column of row of report table
                test_table = [] # reset table
                for i in range(5): # start the loop to get all the validation result
                    table_row = [labels[i]] # set the value of label column
                    table_row.append(res[i+3]) # set the value of precision column
                    table_row.append(res[i*2+3]) # set the value of recall column
                    test_table.append(table_row) # append to report table
                
                print("Best epochs:{} \n\tTrain Accuracy:{}\n\tValid Accuracy:{}\n\tTest  Accuracy:{}\n".format(
                        epochs+1,
                        sess.run(accuracy, feed_dict={x:train, y:label, mode:False}),
                        valid_accuracy,
                        testing_accuracy)
                    ) # print the validation result, which contain current epochs, training set accuracy, validation set accuracy, testing set accuracy
                best_epochs = epochs+1 # Record the best epochs
                save_path = saver.save(sess, "./Models/Team26_HW2.ckpt") #keep the best model
                continuous_fail_count = 0 # reset the fail count
                
            else: # every time the current validation result is not better than the current best valid accuracy
                continuous_fail_count += 1 # we increment the fail count by 1
                if continuous_fail_count > 5: # if total count larger then 5, we consider the model wont impove anymore
                    print("Early Stop! Best batches:{}, Total batches:{}".format(best_epochs,epochs+1)) # print the alert message
                    break  # stop the training step
    
    # Print the finall model performance
    print("Final Report\n")
    print("Validation Result")
    print(tabulate(table, headers=["Label", "Precision", "Recall"]))
    print("Average Accuracy:{}\n".format(best_accuracy))
    
    # render testing performance
    print("Testing Result\n")
    print(tabulate(test_table, headers=["Label", "Precision", "Recall"]))
    print("Average Accuracy:{}\n".format(testing_accuracy))

Best epochs:101 
	Train Accuracy:0.9794921875
	Valid Accuracy:0.9788897633552551
	Test  Accuracy:0.9817085266113281

Best epochs:201 
	Train Accuracy:0.98779296875
	Valid Accuracy:0.9863174557685852
	Test  Accuracy:0.9867678284645081

Best epochs:401 
	Train Accuracy:0.9921875
	Valid Accuracy:0.9878811836242676
	Test  Accuracy:0.9902704954147339

Best epochs:801 
	Train Accuracy:0.99853515625
	Valid Accuracy:0.989835798740387
	Test  Accuracy:0.9918271899223328

Best epochs:1101 
	Train Accuracy:0.99609375
	Valid Accuracy:0.9913995265960693
	Test  Accuracy:0.9914380311965942

Best epochs:1501 
	Train Accuracy:0.99853515625
	Valid Accuracy:0.9925723075866699
	Test  Accuracy:0.9908542633056641

Best epochs:1701 
	Train Accuracy:0.99609375
	Valid Accuracy:0.9933542013168335
	Test  Accuracy:0.9918271899223328

Early Stop! Model seems not improve any more..
Final Report

Validation Result
  Label    Precision    Recall
-------  -----------  --------
      0     1         1
      1     0.9946

## Cross Validation

In [32]:
from sklearn.model_selection import StratifiedShuffleSplit

In [33]:
"""
1. concatenate both X and Y
2. StratifiedShuffleSplit the X and Y to n fold with different propotion
"""
X_con = np.concatenate((X_train1, X_valid1), axis=0) # concatenate X
y_con = np.concatenate((y_train1, y_valid1), axis=0) # concatenate y

print(X.shape)

# create the indice of shuffle split
sss_10_fold = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=0) # indice of 10 fold
sss_5_fold = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0) # indice of 5 fold



(30596, 784)


### 10 fold with dropout

Dropout rate:0.1

In [51]:
total_valid_accuracy = []
total_test_accuracy  = []

labels = ["0","1","2","3","4"]
with tf.Session() as sess:
    for cross_validation_round, (train_index, valid_index) in enumerate(sss_10_fold.split(X_con, y_con)):
        print("Round:{}".format(cross_validation_round+1))
        # Model Hyper Parameter
        epochs_num = 3000
        best_accuracy = 0.8
        best_epochs = 0
        testing_accuracy = None
        continuous_fail_count = 0
        table = []
        test_table = []

        saver = tf.train.Saver() # to store the model

        # re-prepare the training and valid dataset
        train_data = tf.contrib.data.Dataset.from_tensor_slices((X_con[train_index],y_con[train_index])).batch(1024).repeat()
        train_iterator = train_data.make_one_shot_iterator() # Create an iterator to go through the training data
        train_next_batch = train_iterator.get_next()
    
    
        sess.run(tf.global_variables_initializer()) # initialize the weights 
        sess.run(tf.local_variables_initializer()) # initialize the local variables hidden in the tf.metrics.recallmethod.

        for epochs in range(epochs_num): # starting the training process and set the epochs_num
            train, label = sess.run(train_next_batch) # Get the mini-batch data sample
            sess.run(train_step, feed_dict={x:train, y:label, mode:True}) # Feed the features, labe, training_mode  to network to train

            # Get best accuracy
            if epochs % 100 == 0: # validate for every 100 epochs
                res = sess.run([accuracy, precision, recall, prec_zer,prec_one,prec_two,prec_thi,prec_fou,recal_zer,recal_one,recal_two,recal_thi,recal_fou], feed_dict={x:X_con[valid_index], y:y_con[valid_index], mode:False})
                valid_accuracy = res[0]

                if valid_accuracy > best_accuracy: #checking current validation result better than current best one

                    best_accuracy = valid_accuracy # Record the best accuracy
                    best_precision = res[1][0] # Record the overall precision
                    best_recall = res[2][0]# Record the best overall recall

                    # organize each column of row of report table for validation set
                    table = [] # reset table
                    for i in range(5): # start the loop to get all the validation result
                        table_row = [labels[i]] # set the value of label column
                        table_row.append(res[i+3]) # set the value of precision column
                        table_row.append(res[i*2+3]) # set the value of recall column
                        table.append(table_row) # append to report table

                    # organize each column of row of report table for testing set
                    res = sess.run([accuracy, precision, recall, prec_zer,prec_one,prec_two,prec_thi,prec_fou,recal_zer,recal_one,recal_two,recal_thi,recal_fou], feed_dict={x:X_test1, y:y_test1, mode:False})
                    testing_accuracy = res[0]
                    # organize each column of row of report table
                    test_table = [] # reset table
                    for i in range(5): # start the loop to get all the validation result
                        table_row = [labels[i]] # set the value of label column
                        table_row.append(res[i+3]) # set the value of precision column
                        table_row.append(res[i*2+3]) # set the value of recall column
                        test_table.append(table_row) # append to report table
                    best_epochs = epochs+1 # Record the best epochs
                    save_path = saver.save(sess, "./Models/Team26_HW2.ckpt") #keep the best model
                    continuous_fail_count = 0 # reset the fail count

                else: # every time the current validation result is not better than the current best valid accuracy
                    continuous_fail_count += 1 # we increment the fail count by 1
                    if continuous_fail_count > 5: # if total count larger then 5, we consider the model wont impove anymore
                        print("Early Stop! Best batches:{}, Total batches:{}".format(best_epochs, epochs+1)) # print the alert message
                        break  # stop the training step

        # Print the finall model performance
#         print("Final Report\n")
#         print("Validation Result")
#         print(tabulate(table, headers=["Label", "Precision", "Recall"]))
#         print("Average Accuracy:{}\n".format(best_accuracy))

        # render testing performance
        print("Testing Result\n")
        print(tabulate(test_table, headers=["Label", "Precision", "Recall"]))
        print("Average Accuracy:{}\n".format(testing_accuracy))
        
        # Record the performance for each fold
        total_valid_accuracy.append(best_accuracy)
        total_test_accuracy.append(testing_accuracy)
    
    print('\n----------\n Cross Validation Report\n----------\n')
    print('Average Accuracy')
    print('\tValidation set:{}'.format(sum(total_valid_accuracy)/len(total_valid_accuracy)))
    print('\tTesting    set:{}'.format(sum(total_test_accuracy)/len(total_test_accuracy)))

Round:1
Early Stop! Best epochs:1701, Total epochs:2301
Testing Result

  Label    Precision    Recall
-------  -----------  --------
      0     0.989837  0.989837
      1     0.996463  0.983301
      2     0.983301  0.9799
      3     0.991098  0.992952
      4     0.9799    0.992079
Average Accuracy:0.9883245825767517

Round:2
Early Stop! Best epochs:401, Total epochs:1001
Testing Result

  Label    Precision    Recall
-------  -----------  --------
      0     0.99692   0.99692
      1     0.992112  0.98237
      2     0.98237   0.988787
      3     0.981409  0.997357
      4     0.988787  0.993069
Average Accuracy:0.9883245825767517

Round:3
Early Stop! Best epochs:901, Total epochs:1501
Testing Result

  Label    Precision    Recall
-------  -----------  --------
      0     0.989858  0.989858
      1     0.99646   0.983495
      2     0.983495  0.989837
      3     0.993062  0.99207
      4     0.989837  0.992079
Average Accuracy:0.9906596541404724

Round:4
Early Stop! Best epoc

### 5 fold with dropout

Dropout rate:0.1

In [43]:
total_valid_accuracy = []
total_test_accuracy  = []

labels = ["0","1","2","3","4"]
with tf.Session() as sess:
    for cross_validation_round, (train_index, valid_index) in enumerate(sss_5_fold.split(X_con, y_con)):
        print("Round:{}".format(cross_validation_round+1))
        # Model Hyper Parameter
        epochs_num = 3000
        best_accuracy = 0.8
        best_epochs = 0
        testing_accuracy = None
        continuous_fail_count = 0
        table = []
        test_table = []

        saver = tf.train.Saver() # to store the model

        # re-prepare the training and valid dataset
        train_data = tf.contrib.data.Dataset.from_tensor_slices((X_con[train_index],y_con[train_index])).batch(1024).repeat()
        train_iterator = train_data.make_one_shot_iterator() # Create an iterator to go through the training data
        train_next_batch = train_iterator.get_next()


        sess.run(tf.global_variables_initializer()) # initialize the weights 
        sess.run(tf.local_variables_initializer()) # initialize the local variables hidden in the tf.metrics.recallmethod.

        for epochs in range(epochs_num): # starting the training process and set the epochs_num
            train, label = sess.run(train_next_batch) # Get the mini-batch data sample
            sess.run(train_step, feed_dict={x:train, y:label, mode:True}) # Feed the features, labe, training_mode  to network to train

            # Get best accuracy
            if epochs % 100 == 0: # validate for every 100 epochs
                res = sess.run([accuracy, precision, recall, prec_zer,prec_one,prec_two,prec_thi,prec_fou,recal_zer,recal_one,recal_two,recal_thi,recal_fou], feed_dict={x:X_con[valid_index], y:y_con[valid_index], mode:False})
                valid_accuracy = res[0]

                if valid_accuracy > best_accuracy: #checking current validation result better than current best one

                    best_accuracy = valid_accuracy # Record the best accuracy
                    best_precision = res[1][0] # Record the overall precision
                    best_recall = res[2][0]# Record the best overall recall

                    # organize each column of row of report table for validation set
                    table = [] # reset table
                    for i in range(5): # start the loop to get all the validation result
                        table_row = [labels[i]] # set the value of label column
                        table_row.append(res[i+3]) # set the value of precision column
                        table_row.append(res[i*2+3]) # set the value of recall column
                        table.append(table_row) # append to report table

                    # organize each column of row of report table for testing set
                    res = sess.run([accuracy, precision, recall, prec_zer,prec_one,prec_two,prec_thi,prec_fou,recal_zer,recal_one,recal_two,recal_thi,recal_fou], feed_dict={x:X_test1, y:y_test1, mode:False})
                    testing_accuracy = res[0]
                    # organize each column of row of report table
                    test_table = [] # reset table
                    for i in range(5): # start the loop to get all the validation result
                        table_row = [labels[i]] # set the value of label column
                        table_row.append(res[i+3]) # set the value of precision column
                        table_row.append(res[i*2+3]) # set the value of recall column
                        test_table.append(table_row) # append to report table
                    best_epochs = epochs+1 # Record the best epochs
                    save_path = saver.save(sess, "./Models/Team26_HW2.ckpt") #keep the best model
                    continuous_fail_count = 0 # reset the fail count

                else: # every time the current validation result is not better than the current best valid accuracy
                    continuous_fail_count += 1 # we increment the fail count by 1
                    if continuous_fail_count > 5: # if total count larger then 5, we consider the model wont impove anymore
                        print("Early Stop! Best batches:{}, Total batches:{}".format(best_epochs, epochs+1)) # print the alert message
                        break  # stop the training step

        # Print the finall model performance
#         print("Final Report\n")
#         print("Validation Result")
#         print(tabulate(table, headers=["Label", "Precision", "Recall"]))
#         print("Average Accuracy:{}\n".format(best_accuracy))

        # render testing performance
        print("Testing Result\n")
        print(tabulate(test_table, headers=["Label", "Precision", "Recall"]))
        print("Average Accuracy:{}\n".format(testing_accuracy))

        # Record the performance for each fold
        total_valid_accuracy.append(best_accuracy)
        total_test_accuracy.append(testing_accuracy)

    print('\n----------\n Cross Validation Report\n----------\n')
    print('Average Accuracy')
    print('\tValidation set:{}'.format(sum(total_valid_accuracy)/len(total_valid_accuracy)))
    print('\tTesting    set:{}'.format(sum(total_test_accuracy)/len(total_test_accuracy)))

Round:1
Early Stop! Best epochs:2201, Total epochs:2801
Final Report

Validation Result
  Label    Precision    Recall
-------  -----------  --------
      0     0.998305  0.998305
      1     0.996249  0.975042
      2     0.975042  0.983051
      3     0.986939  0.984433
      4     0.983051  0.986134
Average Accuracy:0.9880719184875488

Testing Result

  Label    Precision    Recall
-------  -----------  --------
      0     0.994893  0.994893
      1     0.999108  0.977821
      2     0.977821  0.987792
      3     0.983317  0.986784
      4     0.987792  0.992079
Average Accuracy:0.9887137413024902

Round:2
Early Stop! Best epochs:701, Total epochs:1301
Final Report

Validation Result
  Label    Precision    Recall
-------  -----------  --------
      0     0.989882  0.989882
      1     0.985272  0.979781
      2     0.979781  0.991372
      3     0.979675  0.991846
      4     0.991372  0.982871
Average Accuracy:0.9851307272911072

Testing Result

  Label    Precision    Recall
